# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
csv_file = "exported_weather_df.csv"

weather_df = pd.read_csv(csv_file)

weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Yumen,0,CN,1584426305,9,40.28,97.20,54.84,25.95
1,1,Hilo,90,US,1584426306,78,19.73,-155.09,73.99,5.82
2,2,Necochea,28,AR,1584425745,94,-38.55,-58.74,57.99,4.00
3,3,Rikitea,61,PF,1584426307,84,-23.12,-134.97,80.40,9.53
4,4,Rio Grande,0,BR,1584425769,97,-32.03,-52.10,61.00,8.59
...,...,...,...,...,...,...,...,...,...,...
553,553,Hailar,43,CN,1584426434,86,49.20,119.70,27.30,13.22
554,554,Galle,97,LK,1584426315,78,6.04,80.22,81.84,3.91
555,555,Chimbote,99,PE,1584426435,67,-9.09,-78.58,76.08,3.53
556,556,Ust-Nera,24,RU,1584426435,92,64.57,143.20,-11.22,3.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
# configure gmaps with api key.
gmaps.configure(api_key=g_key)

# store humidity as float, along with taking care of empty cells 
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# store lat and long as floats
locations = weather_df[["Lat", "Long"]].astype(float)

# Create a humidity Heatmap layer
heatmap = gmaps.figure()


heat_mapping = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(weather_df["Humidity"]))

heatmap.add_layer(heat_mapping)

heatmap


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
# new dataframe with narrowed down cities 
filtered_df = weather_df

# drop cells with null values 
filtered_df = filtered_df.dropna()

# filter by temperatures between 70 & 80
filtered_df = filtered_df.loc[(filtered_df["Max Temp"] < 80) & (filtered_df["Max Temp"] > 70)]

# filter to places with wind speed less than 10 mph
filtered_df = filtered_df.loc[filtered_df["Wind Speed"] < 10]

# filter by zero cloudiness 
filtered_df = filtered_df.loc[filtered_df["Cloudiness"] == 0]

filtered_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
39,39,Riyadh,0,SA,1584426059,40,24.69,46.72,71.60,8.05
47,47,Batemans Bay,0,AU,1584426061,63,-35.72,150.18,71.01,3.00
80,80,San Patricio,0,MX,1584426069,79,19.22,-104.70,75.90,0.78
238,238,Eldorado,0,BR,1584426107,91,-23.79,-54.28,71.22,1.52
267,267,La Mira,0,MX,1584426113,86,18.04,-102.33,73.56,3.91
368,368,Sur,0,OM,1584426391,36,22.57,59.53,76.44,5.28
399,399,Nizwá,0,OM,1584426398,38,22.93,57.53,72.66,4.90
473,473,Jiwani,0,PK,1584426416,56,25.05,61.74,73.56,1.86
536,536,Tupã,0,BR,1584426310,91,-21.93,-50.51,71.40,3.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
hotel_name = []
hotel_city = []
hotel_country = []
hotel_lat = []
hotel_long = []

# go through rows of hotel map data frame and iterrate 
for index, row in hotel_map_df.iterrows():
    
    cordz = f"{row["Lat"]},{row["Long"]} 
    
    target_cordz = cordz
    target_radius = 5000
    target_type = "lodging"
    
    params = {
        "location": target_cordz,
        "radius": target_radius,
        "type": target_type, 
        "key": api_keys
    }
               
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params).json()
        
    if response["status"] != "ZERO_RESULTS":
          
        hotel_name.append(response["results"][0]["name"])
        hotel_country.append(response["results"][0]["plus_code"]["compound_code"])
        hotel_city.append(response["results"][0]["vicinity"])
        hotel_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotel_long.append(response["results"][0]["geometry"]["location"]["long"])
        
    else :
        print(f"No hotels meet criteria of {cordz}")
    


SyntaxError: invalid syntax (<ipython-input-90-2e5d9070517b>, line 10)

In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map
